In [1]:
%load_ext autoreload
%autoreload 2

# Generation of turbulence for aircraft simulation

The turbulence is generated using the formula proposed in the documentation of the "Aerospace Blockset" librairy of Matlab (link [here](https://fr.mathworks.com/help/aeroblks/vonkarmanwindturbulencemodelcontinuous.html#mw_2b7e435f-013a-4692-8af3-61f28c7ab1e9)). The turbulence conforms to the MIL-STD-1797A standard.

In [2]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

from turbulence_generator import (
    generate_3d_turbulence,
    generate_perfect_spectra_noise,
    f_u,
    f_v,
    f_w,
    f_p,
    f_q,
    f_r,
    generate_turbulence_times,
    create_turbulence_complete_flight,
    create_turbulence_files,
    create_turbulence_df,
)
from turbulence_parameters import (
    turbulence_std,
    turbulence_scale_length,
    turbulence_bandwidth,
)

## Choice of the turbulence parameters

In [ ]:
plane_parameters = {"b": 13.16}

turbulence_mean_length = 180  # 3 minutes of turbulence
turbulence_part = 0.05  # 5% of turbulences during the flights

turbulence_sampling_freqs = [40]  # Hz

turbulence_levels = [1]  # 1 for light, 2 for moderate, 3 for severe

instruction_dir_path = (
    Path.home()
    / "Documents"
    / "data"
    / "Safire_meghatropique"
    / "flight_instruction_files"
    / "original"
)
print("Flight plan files :")
for file in instruction_dir_path.glob("*.csv"):
    print(file.name)

turbulence_dir_root_path = (
    Path.home()
    / "Documents"
    / "data"
    / "Safire_meghatropique"
    / "turbulence_files"
    / "MIL-STD-1797A"
    / "test"
)

In [ ]:
for turbulence_sampling_freq in turbulence_sampling_freqs:
    for turbulence_level in turbulence_levels:
        turbulence_dir_path = (
            turbulence_dir_root_path
            / (
                "light"
                if turbulence_level == 1
                else "moderate"
                if turbulence_level == 2
                else "severe"
            )
            / f"{turbulence_sampling_freq}Hz"
        )
        print(f"Sampling frequency : {turbulence_sampling_freq}Hz")
        print(f"Turbulence level : {turbulence_level}")

        create_turbulence_files(
            instruction_dir_path,
            turbulence_dir_path,
            plane_parameters,
            turbulence_mean_length=turbulence_mean_length,
            turbulence_part=turbulence_part,
            sampling_frequency=turbulence_sampling_freq,
            turbulence_level=turbulence_level,
        )
        print("Done !\n")

## Illustration of turbulences generated by the script

In [ ]:
f = 40 # turbulence sampling frequency in Hz
V = 100 # aircraft airspeed in m/s
L = 1250 / 3.281 # Turbulence length scale in m
bandwidth = min(100 * V / L / (2*np.pi), f/2) # Turbulence bandwidth in Hz
print(f"bandwidth = {bandwidth} Hz, max Von Karman frequency = {100 * V / L / (2*np.pi)} Hz")
h_m = 1000 # altitude in m
turb_level = 2 # 0: no turbulence, 1: light, 2: moderate, 3: severe
sigma_u, sigma_v, sigma_w = turbulence_std(h_m, turb_level)
Lu, Lv, Lw = turbulence_scale_length(h_m)
parameters = {
    "b": 13.16,
    "V": 100,
    "sigma_u": sigma_u,
    "sigma_v": sigma_v,
    "sigma_w": sigma_w,
    "Lu": Lu,
    "Lv": Lv,
    "Lw": Lw,
    "bandwidth": bandwidth,
}
turb_10240 = generate_perfect_spectra_noise(10240, f, bandwidth, f_p, parameters)
delta_turb_10240 = np.diff(turb_10240) * f
t_max = 3
x = np.linspace(0, t_max, t_max*f)
plt.plot(x, turb_10240[:t_max*f], label="turbulence, m.s^-1")
plt.xlabel('time (s)')
plt.legend()
plt.title(f"First {t_max} seconds of turbulence generation")
plt.show()

### Turbulence spectra

In [ ]:
N = 100
f = 10
fc = 4
noise = generate_perfect_spectra_noise(N, f, fc, f_w, parameters)
frequencies = np.fft.fftfreq(N, 1/f)
plt.plot(frequencies, abs(np.fft.fft(noise)), '+')
plt.xlabel('frequency (Hz)')
plt.ylabel('amplitude')
plt.title(f"Beautiful turbulence spectra")
plt.show()